In [ ]:
from task2.data_man.meta_data import ConllItem
from typing import AnyStr, Dict
from task2.data_man.meta_data import _assign_ner_tags, extract_spans, get_wiki_knowledge, join_tokens, get_wiki_title_knowledge, get_wiki_title_google_type, get_wiki_entities
from task2.data_man.meta_data import read_conll_item_from_file, write_json_gzip
from task2.configuration.config import logging
from task2.configuration import config

In [ ]:
def analyze_coverage(conll_file: AnyStr, entity_vocab: Dict, type: AnyStr='ORG'):
    conll_items = read_conll_item_from_file(conll_file)
    hit_count = 0
    total_count = 0 + 1e-6
    error_conunt = 0
    for item in conll_items:
        gold_spans = extract_spans(item.labels)
        gold_entities = []
        gold_labels = []
        for k in gold_spans:
            if gold_spans[k] == 'O':
                continue
            gold_entities.append(join_tokens(item.tokens[k[0]:k[1]+1])[0])
            gold_labels.append(item.labels[k[0]][2:])
        #logging.info('{}'.format(join_tokens(tokens)[0]))    
        #logging.info('gold entities: {}'.format(gold_entities))
        #logging.info('gold labels: {}'.format(gold_labels))
    
        for i, entity in enumerate(gold_entities):
            if entity in entity_vocab:
                if gold_labels[i] == type:
                    hit_count += 1
                    total_count += 1
                    if type not in entity_vocab[entity]:
                        logging.info(entity)
                        logging.info(entity_vocab[entity])
                        #logging.info(item.id)
                        logging.info('{}'.format(join_tokens(item.tokens)[0]))    
                        error_conunt += 1
            else:
                if gold_labels[i] == type:
                    #logging.info('{}'.format(join_tokens(item.tokens)[0]))    
                    #logging.info('{}: {} is not in'.format(entity, gold_labels[i]))
                    total_count += 1
    logging.info("error count:{}, error_rate: {}%".format(error_conunt, round(error_conunt/total_count, 4) *100))
    return '{}%'.format(round(hit_count / total_count, 4) * 100)
    

In [ ]:
person_entities_vocab = get_wiki_entities(config.wiki_entity_data['person'])

In [ ]:
for k in person_entities_vocab:
    person_entities_vocab[k] = list(set(person_entities_vocab[k]))

In [ ]:
from task2.data_man.meta_data import write_json_gzip
write_json_gzip(config.wiki_entity_data['person'], person_entities_vocab)

In [ ]:

for type in ['Cleric', 'Scientist', 'Artist', 'OtherPER']:
    if type != 'Cleric':
        continue
    coverage = analyze_coverage(config.train_file['Chinese'], person_entities_vocab, type=type)
    logging.info('{} coverage is {}'.format(type, coverage))

In [ ]:
person_entities_vocab['喬佛里·波吉亞']

In [ ]:
import os
import re
import collections
import pandas as pd
group_entities = get_wiki_entities(config.wiki_entity_data['group'])
GROUP_TYPE = {
    'Q4438121': 'sport organization',  
    'Q936518': 'aerospace manufacturer',
    'Q215380': 'musical group',
    'Q32178211': 'musical group',
    'Q891723': 'public corporation',
    'Q7257717': 'public corporation',
    'Q5621421': 'private corparation',
    'Q786820': 'automobile manufacturer',
    'Q18388277': 'technology company',
    'Q43229':'orginazation',
    'Q4830453':'orginazation',
}
type_by_id = collections.defaultdict(list)
for dir in os.walk('group_en_20230106'):

    for l in dir:
        if not isinstance(l, list):
            continue
        for file in l:
            if not file.endswith('csv'):
                continue
            csv_file = './group_en_20230106/{}'.format(file)
            qid = csv_file.split('_')[3]
            data = pd.read_csv(csv_file, sep='\t', header=None)
            type = GROUP_TYPE[qid]
            for row in data.iterrows():
                entity = row[1].values[1]
                if re.match(r'Q[0-9]+', entity):
                    continue
                if entity in group_entities and type == 'orginazation':
                    continue
                if entity in group_entities and type != 'orginazation':
                    if type not in group_entities[entity]:
                        group_entities[entity].append(type)
                if entity not in group_entities:
                    group_entities[entity] = [type]

print(len(group_entities))

In [ ]:
group_entities = get_wiki_entities(config.wiki_entity_data['group'])
from task2.data_man.meta_data import write_json_gzip
#write_json_gzip(config.wiki_entity_data['group'], json_dict=group_entities)

In [ ]:
coverage = analyze_coverage(config.validate_file['English'], group_entities, type='PublicCorp')
logging.info('coverage is {}'.format(coverage))

In [ ]:
import json
with open('med_zh.json', 'r', encoding='utf-8') as f: 
    data = json.load(f)
    write_json_gzip(config.wiki_entity_data['medicine'], data)

In [ ]:
with open('loc_zh.json', 'r', encoding='utf-8') as f: 
    data = json.load(f)
    write_json_gzip(config.wiki_entity_data['location'], data)

In [ ]:
with open('pro_zh.json', 'r', encoding='utf-8') as f: 
    data = json.load(f)
    write_json_gzip(config.wiki_entity_data['product'], data)

In [ ]:
with open('cre_zh.json', 'r', encoding='utf-8') as f: 
    data = json.load(f)
    write_json_gzip(config.wiki_entity_data['creative work'], data)

In [ ]:
data = get_wiki_entities(config.wiki_entity_data['creative work'])

In [ ]:
data = get_wiki_entities(config.wiki_entity_data['creative work'])
coverage = analyze_coverage(config.validate_file['Chinese'], data, type='WrittenWork')
logging.info('coverage is {}'.format(coverage))

In [ ]:
from transformers import *
tokenizer = AutoTokenizer.from_pretrained('bert-base-chinese')

In [ ]:
print(tokenizer("<visual work>")['input_ids'])
print(tokenizer('<medicine>')['input_ids'])

In [ ]:
tokenizer.add_tokens(['visualwork'])

In [ ]:
tokenizer.get_added_vocab()

In [ ]:
import pickle
with open('dict_230111/group_12_language_dict.pkl', 'rb') as f:
    data = pickle.load(f)
group_data = get_wiki_entities(config.wiki_entity_data['group'])
for k in data:
    group_data[k] = data[k]
write_json_gzip(config.wiki_entity_data['group'], group_data)

In [ ]:
with open('dict_230111/location_zh_language_dict.pkl', 'rb') as f:
    data = pickle.load(f)
group_data = get_wiki_entities(config.wiki_entity_data['location'])
for k in data:
    group_data[k] = data[k]
write_json_gzip(config.wiki_entity_data['location'], group_data)

In [ ]:
with open('dict_230111/product_zh_language_dict.pkl', 'rb') as f:
    data = pickle.load(f)
group_data = get_wiki_entities(config.wiki_entity_data['product'])
for k in data:
    group_data[k] = data[k]
write_json_gzip(config.wiki_entity_data['product'], group_data)